# Clustering mobile user movements

Modern cell phones have powerful in-build sensors that allow tracking the physical movements of its user. From the data delivered by the accelerometer we can try to infer the underlying activity, such as walking or standing. Many useful applications can use this information, for instance fitness tracking or adaptive notifications.

In this exercise the task is to infer underlying movements of the user by grouping similar movement using unsupervised techniques. Apart from a small subset, the training samples are not labeled. In the end you may test if your groups correspond to the held out ground-truth activities. 

This task has two key challenges. **Extracting useful features** that are useful for classifying the activities, and implementing **effective clustering methods**. The task has these broad steps:
1. Load and visualize data
3. Extract features
3. Clustering methods
4. Validate clusters

Feel free to optimize your methods in each step and use all you have learned to improve your performance. Also feel free to reorganize your pipeline to make running the all steps at once easier, for instance with `sklearn.pipeline`.

KATE expects your code to define variables with specific names that correspond to certain things we are interested in.

KATE will run your notebook from top to bottom and check the latest value of those variables, so make sure you don't overwrite them.

* Remember to uncomment the line assigning the variable to your answer and don't change the variable or function names.
* Use copies of the original or previous DataFrames to make sure you do not overwrite them by mistake.

You will find instructions below about how to define each variable.

Once you're happy with your code, upload your notebook to KATE to check your feedback.

### Load the data

#### About the data

The experiments have been carried out with a group of volunteers performing six activities (WALKING, WALKINGUPSTAIRS, WALKINGDOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone on the waist ([video](https://www.youtube.com/watch?v=XOEN9W05_4A)). Using its embedded accelerometer and gyroscope, 3-axial linear acceleration, body acceleration and angular velocity were collected.

For more info, the reference paper: [Domain Dataset for Human Activity Recognition Using Smartphones](https://scholar.google.com/scholar?q=a+public+domain+dataset+for+human+activity+recognition+using+smartphones&hl=en&as_sdt=0&as_vis=1&oi=scholart).


#### Import the relevant libraries

In [164]:
import numpy as np
import pandas as pd

#### Import the data files

Each data file has an array of times series, with X, Y and Z directions each.

In [165]:
body_acc_x_train = np.loadtxt("data/body_acc_x_train.txt")
body_acc_y_train = np.loadtxt("data/body_acc_y_train.txt")
body_acc_z_train = np.loadtxt("data/body_acc_z_train.txt")

body_gyro_x_train = np.loadtxt("data/body_gyro_x_train.txt")
body_gyro_y_train = np.loadtxt("data/body_gyro_y_train.txt")
body_gyro_z_train = np.loadtxt("data/body_gyro_z_train.txt")

total_acc_x_train = np.loadtxt("data/total_acc_x_train.txt")
total_acc_y_train = np.loadtxt("data/total_acc_y_train.txt")
total_acc_z_train = np.loadtxt("data/total_acc_z_train.txt")

print(body_acc_x_train.shape)

(7352, 128)


#### We can join the time series into a single raw dataset.

In [166]:
X_train = np.array(np.hstack([body_acc_x_train, body_acc_y_train, body_acc_z_train,
                   body_gyro_x_train, body_gyro_y_train, body_gyro_z_train,
                   total_acc_x_train, total_acc_y_train, total_acc_z_train]))

print(X_train.shape)

(7352, 1152)


#### Load the labels for the first 200 samples of the data. 

The labels indicate:
1. WALKING
2. WALKING_UPSTAIRS
3. WALKING_DOWNSTAIRS
4. SITTING
5. STANDING
6. LAYING

In [167]:
y_train = np.loadtxt("data/y_train.txt")

Print the the number of samples in each category. Is the data set balanced?

In [168]:
y_train_df = pd.DataFrame({"Labels":y_train})
y_train_df.value_counts("Labels")

Labels
5.0    44
1.0    36
4.0    31
6.0    31
3.0    30
2.0    28
Name: count, dtype: int64

#### Visualize the trajectories

Inspect the time series data by plotting some (X,Y,Z) data samples over time.

In [169]:
feature_df = pd.DataFrame()

## Extract features

The raw time-series could be used directly as input to machine learning, but it might not be very effective. A key challenge is to extract features that could help identify and differentiate between activities. Feel free to select from or go beyond the suggestions and come up with your own features.

### Time series properties

* **Magnitude**: Calculate the magnitude of a 3-dimensional time series at each time point, given by the norm of the (X,Y,Z) vector: $$mag(t) = \sqrt{acc_X^2(t) + acc_Y^2(t) + acc_Z^2(t)}$$

* **Jerk**: Calculate the change in acceleration, known as jerk. You may calculate it by the different in acceleration over a time lag: $$jerk(t) = acc(t) - acc(t-lag)$$.

* **Spectral properties**: Calculate the frequency domain analysis using FFT of a time-series.

* **Autoregressive model**: Calculate the auto-regressive model for a time-series and use its parameters as features.

# Valere Notes:

The data consists of 7k rows and 128 cols (time series).

The goal is to calculate measure from the times series. For example the body acceleration magnitude will return magnitude at each time series point. Then for our model we might be interestd in the mean, std, min, max of the magnitude, where each one of these will make up a feature in our model.

Read above the labels correspond to the first 200 samples in the data.

In [170]:
body_acc_magnitude = np.sqrt(body_acc_x_train**2 + body_acc_y_train**2 + body_acc_z_train**2)
body_gyro_magnitude = np.sqrt(body_gyro_x_train**2 + body_gyro_y_train**2 + body_gyro_z_train**2)
total_acc_magnitude = np.sqrt(total_acc_x_train**2 + total_acc_y_train**2 + total_acc_z_train**2)

#### Join time series

In [171]:
data_dict = {
    "body_acc_x_train": body_acc_x_train,
    "body_acc_y_train": body_acc_y_train,
    "body_acc_z_train": body_acc_z_train,
    "body_gyro_x_train": body_gyro_x_train,
    "body_gyro_y_train": body_gyro_y_train,
    "body_gyro_z_train": body_gyro_z_train,
    "total_acc_x_train": total_acc_x_train,
    "total_acc_y_train": total_acc_y_train,
    "total_acc_z_train": total_acc_z_train,
    "body_acc_magnitude": body_acc_magnitude,
    "body_gyro_magnitude": body_gyro_magnitude,
    "total_acc_magnitude": total_acc_magnitude
}

In [172]:
# from statsmodels.tsa.ar_model import AutoReg

# def calculate_ar_params(series, lag):
#     ar_params = []
#     for i in series:
#         model = AutoReg(i, lags=lag).fit()
#         ar_params.append(model.params)
#     return np.array(ar_params)

# ar_dict = pd.DataFrame()
# for key, value in data_dict.items():
#     ar_params = calculate_ar_params(value, 1)
#     ar_params = pd.DataFrame(ar_params)
#     ar_params.columns = [key + "_ar_1", key + "_ar_2"]
#     ar_dict = pd.concat([ar_dict, ar_params], axis=1)



In [173]:
fft = {}
for key, value in data_dict.items():
    fft[key +"FFT"] = np.fft.fft(value)

temp = {}
for key, value in fft.items():
    temp[key +"_mean"] = np.mean(value, axis = 1)
    temp[key +"_std"] = np.std(value, axis = 1)
    temp[key +"_min"] = np.min(value, axis = 1)
    temp[key +"_max"] = np.max(value, axis = 1)
    temp[key +"_median"] = np.median(value, axis = 1)

fft_features = pd.DataFrame(temp)

### Statistical features

* Design statistical features to extract from the time-series you have gathered, such as minimal/maximal value or standard deviation. Again, you can be creative on features that might be relevant to classify a time series, for instance the Median Absolute Deviation or the Signal Magnitude Area.

In [174]:
def IQR(dist, axis = 1):
    return np.percentile(dist, 75) - np.percentile(dist, 25)

funcs = [np.min, np.std, np.mean, np.max, IQR]


for key, value in data_dict.items():
    for func in funcs:
        feature_df[key + "_" + func.__name__] = func(value, axis=1)

In [175]:
X_train = pd.concat([feature_df, fft_features], axis = 1)

* Implement a function that estimate these statistical features for each time series and join them into a training dataset.

How many features do you have in total?

In [176]:
# How many features do you have in total?
print(X_train.shape)

(7352, 144)


## Clustering activities

Now that you have defined your features, the second key challenge is to model your data with clustering algorithms, trying to group data sample of the same activity together.

### Preprocess features

Make sure to preprocess your data to be adequate as input to your algorithms.

In [177]:

# Assuming X_train is your training data
if np.iscomplexobj(X_train):
    X_train = np.real(X_train)

In [178]:
# ### Preprocess features

# Make sure to preprocess your data to be adequate as input to your algorithms.
# Manual feautre scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# def min_max_scale(series):
#     return (series - series.min()) / (series.max() - series.min())

# X_train = X_train.apply(min_max_scale, axis = 0)


## Clustering

* Now is the time to implement the core clustering algorithm with the data you have preprocessed.
* You can use any algorithm you have learned. Remember to optimize its parameters for performance.
* Note that you can use the few available labels to extend or evaluate your model.

In [179]:
# ## Clustering

# * Now is the time to implement the core clustering algorithm with the data you have preprocessed.
# * You can use any algorithm you have learned. Remember to optimize its parameters for performance.
# * Note that you can use the few available labels to extend or evaluate your model.

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(X_train)

KMeans(n_clusters=6, random_state=42)

## Evaluate your clusters

Now we can try to visualize how our samples were grouped. As usual, it's not simple to visualize high dimensional data, so we will focus on a couple of dimensions.

### Visualize ground-truth groups

Plot a scatter plot of two features of your choice for all samples in a single colour. Overlay it with scatter plot of the labeled samples, coloring them by their label.

Do the activities seem well separated? Which seem similar?

In [180]:
# import seaborn as sns

# truth_df = pd.DataFrame(X_train[0:200])

# sns.scatterplot(x = truth_df[0], y = truth_df[1], hue = y_train)

### Visualize your groups

Plot a scatter plot of two features of your choice for all samples, coloring them by their cluster.

Do your cluster seem to correspond to the true activity clusters?

Print the size of each cluster. Does it seem balanced? What does it mean?

### Performance metrics

#### Confusion matrix

We can evaluate the clusters by checking if they can be mapped into activities for the labeled samples. One way to inspect this mapping is through the confusion matrix. 

Calculate and print the confusion matrix for your clustering labels and the ground truth for the samples that are labeled.

Interpret the result by analyzing which types of activities each cluster contains. Note that if the matrix element $C_{i,j} = 8$, it means your model called 8 samples of category $j$ as being $i$.

In [181]:
perf = kmeans.labels_[0:200]

confusion_matrix = pd.crosstab(y_train, perf, rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1,2,3,4,5
Actual,,,,,,
1.0,24,0,4,2,6,0
2.0,20,0,0,5,3,0
3.0,2,0,6,4,18,0
4.0,0,29,0,0,0,2
5.0,0,44,0,0,0,0
6.0,1,1,0,0,0,29


#### Adjusted Rand Score (ARS)

A common metric to evaluate if a clustering maps into some grouping is the adjusted Rand score. The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings. The raw RI score is then adjusted for chance into the adjusted score.

Calculate the ARS for the labeled samples, using `adjusted_rand_score()` in `sklearn`. This is the performance metric that will be used to test your predictions in the test set. A score over 0.5 is a good target. Can you do better?

In [182]:
# Calculate adjusted rand score for clusters
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(y_train, perf)

0.5120853294433937

For comparison, run your clustering model directly on the raw time-series data `X_train` (without feature extraction and preprocessing) and check the performance.

### Predict test set groups

Save the clustering labels for all samples in the variable `cluster_labels`. It will be evaluated against the ground-truth using the adjusted rand score.

In [183]:
cluster_labels = kmeans.labels_